https://www.youtube.com/watch?v=KYFfMymTspE

In [8]:
from keras.applications.vgg16 import VGG16
from keras.models import Sequential,Model 
from keras.layers import Dense,Dropout,Activation,Flatten,GlobalAveragePooling2D
from keras.layers import Conv2D,MaxPooling2D,ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K

In [9]:
img_rows,img_cols = 128,128

VGG = VGG16(weights='imagenet',include_top=False,input_shape=(img_rows,img_cols,3))

for layer in VGG.layers:
    layer.trainable=False

def addTopModelVGG(bottom_model, num_classes):

    top_model = bottom_model.output
    top_model = GlobalAveragePooling2D()(top_model)
    top_model = Dense(512,activation='relu')(top_model)
    top_model = Dense(256,activation='relu')(top_model)
    top_model = Dense(256,activation='relu')(top_model)
    top_model = Dense(128,activation='relu')(top_model)
    top_model = Dense(128,activation='relu')(top_model)
    top_model = Dense(num_classes,activation='softmax')(top_model)

    return top_model

num_classes = 3

FC_Head = addTopModelVGG(VGG,num_classes)

model = Model(inputs = VGG.input, outputs = FC_Head)

print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 128, 128, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 128, 128, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 128, 128, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 64, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 64, 64, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 64, 64, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 32, 32, 128)       0         
__________

In [10]:
train_data_dir = 'train'
validation_data_dir = 'test'

train_datagen = ImageDataGenerator(
                    rescale=1./255,
                    rotation_range=30,
                    width_shift_range=0.3,
                    height_shift_range=0.3,
                    horizontal_flip=True,
                    fill_mode='nearest'
                                   )

validation_datagen = ImageDataGenerator(rescale=1./255)

batch_size = 32

train_generator = train_datagen.flow_from_directory(
                        train_data_dir,
                        target_size = (img_rows,img_cols),
                        batch_size = batch_size,
                        class_mode = 'categorical'
                        )

validation_generator = validation_datagen.flow_from_directory(
                            validation_data_dir,
                            target_size=(img_rows,img_cols),
                            batch_size=batch_size,
                            class_mode='categorical')

from keras.optimizers import RMSprop,Adam
from keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau

checkpoint = ModelCheckpoint(
                             'finger_sign_vgg_19.h5',
                             monitor='val_loss',
                             mode='min',
                             save_best_only=True,
                             verbose=1)

earlystop = EarlyStopping(
                          monitor='val_loss',
                          min_delta=0,
                          restore_best_weights=True,
                          patience=10,
                          verbose=1)

learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss', 
                                            patience=5, 
                                            verbose=1, 
                                            factor=0.2, 
                                            min_lr=0.0001)

callbacks = [earlystop,checkpoint,learning_rate_reduction]

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=0.001),
              metrics=['accuracy']
              )

nb_train_samples = 1200
nb_validation_samples = 300  

epochs = 10
batch_size = 32

history = model.fit_generator(
            train_generator,
            steps_per_epoch=nb_train_samples//batch_size,
            epochs=epochs,
            callbacks=callbacks,
            validation_data=validation_generator,
            validation_steps=nb_validation_samples//batch_size)


Found 600 images belonging to 3 classes.
Found 150 images belonging to 3 classes.
Epoch 1/10
37/37 [==============================] - 287s 8s/step - loss: 1.0827 - acc: 0.4136 - val_loss: 0.8870 - val_acc: 0.5899

Epoch 00001: val_loss improved from inf to 0.88705, saving model to finger_sign_vgg_19.h5
Epoch 2/10
37/37 [==============================] - 239s 6s/step - loss: 0.8712 - acc: 0.5146 - val_loss: 0.6842 - val_acc: 0.6381

Epoch 00002: val_loss improved from 0.88705 to 0.68420, saving model to finger_sign_vgg_19.h5
Epoch 3/10
37/37 [==============================] - 255s 7s/step - loss: 0.6792 - acc: 0.6163 - val_loss: 0.5327 - val_acc: 0.6940

Epoch 00003: val_loss improved from 0.68420 to 0.53268, saving model to finger_sign_vgg_19.h5
Epoch 4/10
37/37 [==============================] - 247s 7s/step - loss: 0.6194 - acc: 0.6601 - val_loss: 0.5582 - val_acc: 0.7687

Epoch 00004: val_loss did not improve from 0.53268
Epoch 5/10
37/37 [==============================] - 245s 7s/s